In [1]:
import pandas as pd
import numpy

In [2]:
data_file = 'data/wikitext-2/train.txt'

In [3]:
window_size = 7

def get_tokens(data_file):
    with open(data_file) as d:
        for line in d:
            line = line.strip()
            if len(line) > 0:
                l = []
                for split_at in range(0, len(line), window_size):
                    l.append(line[split_at:split_at+window_size])
                yield l
                

In [12]:
from collections import Counter
candidates = Counter()

context_window_size = 3



for line in get_tokens(data_file):
    print(line)
    for t in range(len(line)):
        tok = line[t]
        context_left_size = max(t - context_window_size, 0)
        context_right_size = min(t + 1 + context_window_size, len(line))
        context_left = ''.join(line[context_left_size:t])
        context_right = ''.join(line[t + 1: context_right_size])
        #print(context_left, tok, context_right)
        for i in range(len(tok)):
            for j in range(i, len(tok)):
                right_context = tok[j+1:] + context_right
                left_context =  context_left + tok[:i]
                total_context = left_context + right_context
                candidate = tok[i:j+1]
                for c in total_context:
                    print(candidate, c)
                    if candidate not in candidates:
                        candidates[candidate] = Counter()
                    candidates[candidate][c] += 1
                    print(candidates[candidate])
                print(left_context, '+', candidate, '+', right_context)
            print("-")
        print()
    break

['= Valky', 'ria Chr', 'onicles', ' III =']
=  
Counter({' ': 1})
= V
Counter({' ': 1, 'V': 1})
= a
Counter({' ': 1, 'V': 1, 'a': 1})
= l
Counter({' ': 1, 'V': 1, 'a': 1, 'l': 1})
= k
Counter({' ': 1, 'V': 1, 'a': 1, 'l': 1, 'k': 1})
= y
Counter({' ': 1, 'V': 1, 'a': 1, 'l': 1, 'k': 1, 'y': 1})
= r
Counter({' ': 1, 'V': 1, 'a': 1, 'l': 1, 'k': 1, 'y': 1, 'r': 1})
= i
Counter({' ': 1, 'V': 1, 'a': 1, 'l': 1, 'k': 1, 'y': 1, 'r': 1, 'i': 1})
= a
Counter({'a': 2, ' ': 1, 'V': 1, 'l': 1, 'k': 1, 'y': 1, 'r': 1, 'i': 1})
=  
Counter({' ': 2, 'a': 2, 'V': 1, 'l': 1, 'k': 1, 'y': 1, 'r': 1, 'i': 1})
= C
Counter({' ': 2, 'a': 2, 'V': 1, 'l': 1, 'k': 1, 'y': 1, 'r': 1, 'i': 1, 'C': 1})
= h
Counter({' ': 2, 'a': 2, 'V': 1, 'l': 1, 'k': 1, 'y': 1, 'r': 1, 'i': 1, 'C': 1, 'h': 1})
= r
Counter({' ': 2, 'a': 2, 'r': 2, 'V': 1, 'l': 1, 'k': 1, 'y': 1, 'i': 1, 'C': 1, 'h': 1})
= o
Counter({' ': 2, 'a': 2, 'r': 2, 'V': 1, 'l': 1, 'k': 1, 'y': 1, 'i': 1, 'C': 1, 'h': 1, 'o': 1})
= n
Counter({' ': 2, 'a'